In [1]:
#### Import Library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity,calculate_kmo
import xgboost as xgb
from sklearn.model_selection import train_test_split,GridSearchCV,KFold
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score, recall_score, f1_score,ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
import random
from collections import Counter
from sklearn.tree import DecisionTreeClassifier
import itertools
import os
import cupy as cp

In [2]:
# Get the path of the file
# Current directiory for Jupyter
script_dir = os.getcwd() 

# Check the path whether correct 
print(script_dir)

#Buld corresponding path
file_path = os.path.join(script_dir, "Data")
Original_File_path = os.path.join(file_path, "Original_Data.csv")
Data_Without_Outlier_Path = os.path.join(file_path, "Data_Without_Outlier.csv")

# Test whether path correct
print("Original File Path:", Original_File_path)
print("Data Without Outlier Path:", Data_Without_Outlier_Path)


# Read the csv files
original_data = pd.read_csv(Original_File_path,encoding='utf-8')
data_without_outlier = pd.read_csv(Data_Without_Outlier_Path,encoding='utf-8')

# Check the csv file
print(original_data.head())
print(data_without_outlier.head())

# Check the data
original_data.describe()

C:\Users\judyw\Desktop\FINAL\WashU\ESE_417
Original File Path: C:\Users\judyw\Desktop\FINAL\WashU\ESE_417\Data\Original_Data.csv
Data Without Outlier Path: C:\Users\judyw\Desktop\FINAL\WashU\ESE_417\Data\Data_Without_Outlier.csv
   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3     

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [2]:
# the decision tree algorithm
class node:
    ## Initial the Node parameter
    def __init__(self, feature=None, bar=None, left=None, right=None, value=None):
        self.feature = feature
        self.bar = bar
        self.left = left
        self.right = right
        self.value = value
# Create class for building descision Tree
class decisiontree:
    def __init__(self,max_depth, min_samples_split):
        self.max_depth=max_depth
        self.min_samples_split=min_samples_split
        self.root=None
    # Trainning mode process
    def fit(self,x,y):
        # we apply grow_tree build decision tree
        self.root=self.grow_tree(x,y)
    # find left and right leaf for every bar
    def split_before(self,x_col,bar):
        left_ind=np.where(x_col<=bar)[0]
        right_ind=np.where(x_col>bar)[0]
        if len(left_ind)==0 or len(right_ind)==0:
            return None, None
        return left_ind, right_ind
    # Calculate the entropy
    def entropy(self,x):
        count=np.bincount(x)
        temp=count/len(x)
        sum=-np.sum([p*np.log2(p) for p in temp if p>0])
        return sum
    # Used to calculate information gain
    def gain(self,y,left,right):
        if left is None or right is None:
            return 0
        total_en=self.entropy(y)
        n=len(y)
        n_left,n_right=len(left),len(right)
        if n_left==0 or n_right==0:
            return 0
        minus=(n_left/n)*self.entropy(y[left])+(n_right/n)*self.entropy(y[right])
        # Information gain = total entropy - weighted entropy of left and right subsets
        return total_en-minus
    def split(self,x,y):
        best_gain=-1
        # Initial the features and bars
        best_feature, best_bar=None,None
        # Iterate all features and calculate depend on the bar and get the index
        for i in range(x.shape[1]):
            x_col=x[:,i]
            bar=np.unique(x_col)
            for j in bar:
                left_ind,right_ind=self.split_before(x_col,j)
                # Call the gain function to calculate the entropy
                gain=self.gain(y,left_ind,right_ind)
                # The best split point is the biggest entropy
                if gain>best_gain:
                    best_feature=i
                    best_bar=j
                    best_gain=gain
        return best_feature,best_bar
                
    def grow_tree(self,x,y,dep=0):
        n_sample, n_feature=x.shape
        # When stop grow: 
        """
         1. reach the max depth
         2. the # of samples < # of split samples or lables
        """
        if dep>=self.max_depth or n_sample<self.min_samples_split or len(np.unique(y)) == 1:
            leaf_label=np.bincount(y).argmax()
            return node(value=leaf_label)
        # how to split
        feature,bar=self.split(x,y)
        if feature is None:
            leaf_label=np.bincount(y).argmax()
            return node(value=leaf_label)
        left_ind,right_ind=self.split_before(x[:,feature],bar)
        if left_ind is None or right_ind is None:
            return node(value=np.bincount(y).argmax())
        left_tree=self.grow_tree(x[left_ind],y[left_ind],dep+1)
        right_tree=self.grow_tree(x[right_ind],y[right_ind],dep+1)
        return node(feature=feature,bar=bar,left=left_tree,right=right_tree)
    # It will be used to predict the single sample
    def find_value(self,x,node):
        if node.value is not None:
            return node.value
        if x[node.feature]<=node.bar:
            return self.find_value(x,node.left)
        if x[node.feature]>node.bar:
            return self.find_value(x,node.right)
    # Predict whole dataset
    def predict(self,X):
        return np.array([self.find_value(x,self.root) for x in X])
        

In [4]:
# the random forest algorithm
class randomforest:
    # Initial the random forest parameter
    def __init__(self,n_tree,max_depth,min_sample_split,random_seed):
        self.n_tree=n_tree  #numbers of trees
        self.max_depth=max_depth  # the max depth of every tree
        self.min_sample_split=min_sample_split # below this number node cannot be splited
        self.random_seed=random_seed
        self.trees=[]
    # Train the random forest model
    def fit(self,x,y):
        x=np.array(x)
        y=np.array(y)
        self.trees=[]
        np.random.seed(self.random_seed)
        for _ in range(self.n_tree):
            # bootstrap
            indexs=np.random.choice(len(x),len(x),replace=True)
            x_selected=x[indexs]
            y_selected=y[indexs]
            # train every tree
            tree=DecisionTreeClassifier(max_depth=self.max_depth, min_samples_split=self.min_sample_split,random_state=self.random_seed,class_weight='balanced')
            #tree=decisiontree(max_depth=self.max_depth,min_samples_split=self.min_sample_split)
            tree.fit(x_selected,y_selected)
            self.trees.append(tree)
            #print('done')
    # prediction based on the random forest model
    def predict(self,x):
        x=np.array(x,dtype=np.float64)
        prediction=np.array([tree.predict(x) for tree in self.trees])
        prediction = prediction.astype(int)
        return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=prediction)




In [5]:
#k-forth validation on dataset_without outliers
X=data_without_outlier.iloc[:,0:11]
y=data_without_outlier['quality']
param_grid={ # List the paramters we want to explore
    'n_tree' :[100,200,300,400,500],
    'max_depth' : [10,20,30,40],
    'min_sample_split' : [3,5,7,9],
    'random_seed' : [50,60,70,80]}
# K-fold process
kf = KFold(n_splits=5, shuffle=True, random_state=42)
best_k=None
best_sc=-1
# Split the data
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
mean_store=[]
# Iterate the num of tree
for n_tr in param_grid['n_tree'] :
    # Iterate the each depeth
    for max_d in param_grid['max_depth']:
        # Iterate the min sample batch
        for min_s in param_grid['min_sample_split']:
            # Build the random seed
            for ran in param_grid['random_seed']:
                # The set used to store the scores
                scors=[]
                # Based on K - fold algorithm
                for train_ind, val_ind in kf.split(x_train):
                    # Train and test set
                    x_fold_train, x_fold_val = x_train.iloc[train_ind], x_train.iloc[val_ind]
                    y_fold_train, y_fold_val = y_train.iloc[train_ind], y_train.iloc[val_ind]
                    model=randomforest(n_tree=n_tr,max_depth=max_d,min_sample_split=min_s,random_seed=ran)
                    model.fit(x_fold_train,y_fold_train)
                    y_pred=model.predict(x_fold_val)
                    score=accuracy_score(y_fold_val, y_pred)
                    scors.append(score)
                mean=np.mean(scors)
                mean_store.append(mean)
                print(f"Params: n_tree={n_tr}, max_depth={max_d}, min_samples_split={min_s}, random_seed={ran},Accuracy: {mean:.4f}")
                if mean>best_sc:
                    best_sc=mean
                    best_k={'n_tree' :n_tr,  
                           'max_depth': max_d,
                           'min_sample_split' :min_s,
                           'random_seed' : ran}


print("Best Parameters:", best_k)
print("Best Cross-Validation Accuracy:", best_sc)
# Try the best 
best_model=randomforest(n_tree=best_k['n_tree'],max_depth=best_k['max_depth'],min_sample_split=best_k['min_sample_split'],random_seed=best_k['random_seed'])
best_model.fit(x_train, y_train)
y_test_pre=best_model.predict(x_test)
test_accuracy = accuracy_score(y_test, y_test_pre)
print(f"Test Accuracy: {test_accuracy:.4f}")

Params: n_tree=100, max_depth=10, min_samples_split=3, random_seed=50,Accuracy: 0.6794
Params: n_tree=100, max_depth=10, min_samples_split=3, random_seed=60,Accuracy: 0.6864
Params: n_tree=100, max_depth=10, min_samples_split=3, random_seed=70,Accuracy: 0.6716
Params: n_tree=100, max_depth=10, min_samples_split=3, random_seed=80,Accuracy: 0.6692
Params: n_tree=100, max_depth=10, min_samples_split=5, random_seed=50,Accuracy: 0.6786
Params: n_tree=100, max_depth=10, min_samples_split=5, random_seed=60,Accuracy: 0.6778
Params: n_tree=100, max_depth=10, min_samples_split=5, random_seed=70,Accuracy: 0.6645
Params: n_tree=100, max_depth=10, min_samples_split=5, random_seed=80,Accuracy: 0.6669
Params: n_tree=100, max_depth=10, min_samples_split=7, random_seed=50,Accuracy: 0.6755
Params: n_tree=100, max_depth=10, min_samples_split=7, random_seed=60,Accuracy: 0.6763
Params: n_tree=100, max_depth=10, min_samples_split=7, random_seed=70,Accuracy: 0.6638
Params: n_tree=100, max_depth=10, min_sampl

In [7]:
# grid-search on dataset without outliers
X=data_without_outlier.iloc[:,0:11]
y=data_without_outlier['quality']
param_grid={
    'n_tree' :[100,200,300,400,500],
    'max_depth' : [10,20,30,40],
    'min_sample_split' : [3,5,7,9],
    'random_seed' : [50,60,70,80]}

best_k=None
best_sc=-1
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
ac_store=[]
f1_store=[]
for n_tr in param_grid['n_tree'] :
    for max_d in param_grid['max_depth']:
        for min_s in param_grid['min_sample_split']:
            for ran in param_grid['random_seed']:
                model=randomforest(n_tree=n_tr,max_depth=max_d,min_sample_split=min_s,random_seed=ran)
                model.fit(x_train, y_train)
                y_pre=model.predict(x_test)
                score_1=accuracy_score(y_test, y_pre)
                score_2=f1_score(y_test, y_pre,average='weighted')
                ac_store.append(score_1)
                f1_store.append(score_2)
                print(f"Params: n_tree={n_tr}, max_depth={max_d}, min_samples_split={min_s}, random_seed={ran},Accuracy: {score_1:.4f},F1-score={score_2:.4f}")
                #print(classification_report(y_test, y_pre, target_names=['3','4','5','6','7','8']))
                if score_1>best_sc:
                    best_sc=score_1
                    best_k={'n_tree' :n_tr,  
                           'max_depth': max_d,
                           'min_sample_split' :min_s,
                           'random_seed' : ran}


print("Best Parameters:", best_k)
print("Best Cross-Validation Accuracy:", best_sc)

Params: n_tree=100, max_depth=10, min_samples_split=3, random_seed=50,Accuracy: 0.6625,F1-score=0.6526
Params: n_tree=100, max_depth=10, min_samples_split=3, random_seed=60,Accuracy: 0.6719,F1-score=0.6615
Params: n_tree=100, max_depth=10, min_samples_split=3, random_seed=70,Accuracy: 0.6719,F1-score=0.6617
Params: n_tree=100, max_depth=10, min_samples_split=3, random_seed=80,Accuracy: 0.6344,F1-score=0.6227
Params: n_tree=100, max_depth=10, min_samples_split=5, random_seed=50,Accuracy: 0.6594,F1-score=0.6517
Params: n_tree=100, max_depth=10, min_samples_split=5, random_seed=60,Accuracy: 0.6656,F1-score=0.6563
Params: n_tree=100, max_depth=10, min_samples_split=5, random_seed=70,Accuracy: 0.6656,F1-score=0.6597
Params: n_tree=100, max_depth=10, min_samples_split=5, random_seed=80,Accuracy: 0.6500,F1-score=0.6384
Params: n_tree=100, max_depth=10, min_samples_split=7, random_seed=50,Accuracy: 0.6625,F1-score=0.6541
Params: n_tree=100, max_depth=10, min_samples_split=7, random_seed=60,Acc